# 集成算法原理：
TODO

# xgboost

In [1]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score

In [6]:
iris = datasets.load_iris()
data = iris.data
label = iris.target

print(data.shape, len(set(label)))
# 150 个样本，4个特征，3个类别

(150, 4) 3


In [7]:
########### 便于特征工程 ########################
data1 = pd.DataFrame(data)
data1.columns = ['sepal_l', 'sepal_w', 'petal_l', 'petal_w']
print(data1.head())

   sepal_l  sepal_w  petal_l  petal_w
0      5.1      3.5      1.4      0.2
1      4.9      3.0      1.4      0.2
2      4.7      3.2      1.3      0.2
3      4.6      3.1      1.5      0.2
4      5.0      3.6      1.4      0.2


In [8]:
label1 = pd.DataFrame(label)
label1.columns = ['label']
print(label1.head())

   label
0      0
1      0
2      0
3      0
4      0


In [19]:
########### 拆分数据集 ##############################
train_x, test_x, train_y, test_y = train_test_split(
    data1.values,
    label1.values, # 数组形状：105,1
    test_size=0.3,
    random_state=42,
)
print("训练集长度：", len(train_x), '\n', "测试集长度：", len(test_y))

训练集长度： 105 
 测试集长度： 45


In [13]:
!pip install xgboost

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 127.6 MB 11.0 MB/s eta 0:00:01


In [15]:
########## xgboost 原生接口 ########################
import xgboost as xgb

# 验证集，数据需要转换成 DMatrix 对象
test_data = xgb.DMatrix(test_x, label=test_y)

# 超参数
xgb_params = {
    'eta': 0.3,                        # 学习率
    'silent': True,                    # 输出运行信息
    'objective': 'multi:softprob',     # 使用多分类生成概率矩阵格式
    'num_class': 3,                    # 类别数
    'max_depth': 3                     # 深度
}
num_rounds = 20                        # 训练轮次

# 训练模型，输入数据 DMatrix 对象
model = xgb.train(xgb_params, xgb.DMatrix(train_x, label=train_y), num_rounds)

# 模型预测
test_pred = model.predict(test_data)
print(test_pred[:5])

[11:17:11] WARNING: /workspace/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[[0.00650657 0.96226174 0.03123166]
 [0.970643   0.02533227 0.00402478]
 [0.0033913  0.00692109 0.9896876 ]
 [0.00654362 0.9677424  0.02571394]
 [0.00615641 0.9104776  0.08336602]]


In [16]:
########## 模型性能 #########################################
y_hat = np.asarray([np.argmax(row) for row in test_pred])
print('预测结果：', y_hat)
print("验证集准确率：", precision_score(test_y, y_hat, average='macro'))
print("验证集召回率：", recall_score(test_y, y_hat, average='macro'))

预测结果： [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集准确率： 1.0
验证集召回率： 1.0


# LightGBM

In [17]:
!pip install lightgbm

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     |████████████████████████████████| 1.2 MB 3.0 MB/s eta 0:00:01


In [27]:
# 划分训练集和测试集
train_x, test_x, train_y, test_y = train_test_split(
    data1,
    label1,
    test_size=0.3,
    random_state=42,
)
print("训练集长度:", len(train_x))
print("测试集长度:", len(test_x))

训练集长度: 105
测试集长度: 45


In [30]:
########## lightgbm 原生接口 ########################
import lightgbm as lgb

# 数据转换成模块要求的格式
train_data = lgb.Dataset(train_x, train_y)
test_data = lgb.Dataset(test_x, test_y)

# 设置参数
lgb_params = {
    'boosting_type': 'gbdt',
    'objective': 'multiclass',
    'metric': 'multi_error',
    'verbose': 1,  # <0 显示致命的, =0 显示错误 (警告), >0 显示信息
    'num_class': 3  # 类别数
}

# 模型训练
clf = lgb.train(
    lgb_params,
    train_data,
    num_boost_round=10,  # 训练轮次
    valid_sets=[train_data, test_data],
    verbose_eval=1,
)
# 模型预测
test_pre = clf.predict(
    test_x,
    num_iteration=clf.best_iteration,  # 最佳的模型
)
# print(test_pre)
print(test_pre[:5])

# 选择表示最高概率的列
test_pre_1 = np.asarray([np.argmax(row) for row in test_pre])
print("test的预测结果:", test_pre_1)

# 模型评估
print('验证集精准率：', precision_score(test_y, test_pre_1, average='macro'))
print('验证集召回率：', recall_score(test_y, test_pre_1, average='macro'))

[1]	training's multi_error: 0.133333	valid_1's multi_error: 0.155556
[2]	training's multi_error: 0.0571429	valid_1's multi_error: 0.0444444
[3]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[4]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[5]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[6]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[7]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[8]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[9]	training's multi_error: 0.0761905	valid_1's multi_error: 0
[10]	training's multi_error: 0.0666667	valid_1's multi_error: 0
[[0.13683286 0.63500393 0.22816321]
 [0.69436834 0.15467706 0.15095461]
 [0.12934308 0.16125127 0.70940565]
 [0.14172417 0.62195656 0.23631927]
 [0.13683286 0.63500393 0.22816321]]
test的预测结果: [1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


In [31]:
############## sklearn 接口： ########################################

from lightgbm import LGBMClassifier

# 模型参数：
lgb_params = {
    'learning_rate': 0.1,
    'max_bin': 150,             # 分箱操作
    'num_leaves': 32,           # 叶子树   
    'max_depth': 11,            # 最大深度
    'objective': 'multiclass',
    'n_estimators': 300         # 迭代次数
}
model = LGBMClassifier(**lgb_params)

# Numpy 数组格式的数据直接输入
model.fit(train_x, train_y)

# 预测，输出预测结果
y_hat = model.predict(test_x)
print(y_hat)

# 模型评估
print('验证集精准率：', precision_score(test_y, y_hat, average='macro'))
print('验证集召回率：', recall_score(test_y, y_hat, average='macro'))

[1 0 2 1 1 0 1 2 1 1 2 0 0 0 0 1 2 1 1 2 0 2 0 2 2 2 2 2 0 0 0 0 1 0 0 2 1
 0 0 0 2 1 1 0 0]
验证集精准率： 1.0
验证集召回率： 1.0


/home/yangbin7/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/yangbin7/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


> - lgb.train 中正则化参数为 `lambda_l1, lambda_l2`，sklearn 中对应为 `reg_alpha,reg_lambda`
- 多分类时 `lgb.train` 除了 `objective:multiclass` 外，还要指定 `num_class:5`，而 sklearn 则只需要指定`objective:multiclass`
- 迭代次数在sklearn 中是 `n_estimators:20` ，在模型初始化时指定

# 银行客服分类
「银行营销数据集(Bank Marketing Data Set)」，「分类目标」是预测客户是(' 1 ')或者否(' 0 ')购买该银行的产品。

In [33]:
import numpy as bp
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score
import lightgbm as lgb

## 数据集

In [34]:
path = '../datasets/'
train = pd.read_csv(path + 'bank_train_set.csv')
test = pd.read_csv(path + 'bank_test_set.csv')

In [36]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB


In [37]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         10852 non-null  int64 
 1   age        10852 non-null  int64 
 2   job        10852 non-null  object
 3   marital    10852 non-null  object
 4   education  10852 non-null  object
 5   default    10852 non-null  object
 6   balance    10852 non-null  int64 
 7   housing    10852 non-null  object
 8   loan       10852 non-null  object
 9   contact    10852 non-null  object
 10  day        10852 non-null  int64 
 11  month      10852 non-null  object
 12  duration   10852 non-null  int64 
 13  campaign   10852 non-null  int64 
 14  pdays      10852 non-null  int64 
 15  previous   10852 non-null  int64 
 16  poutcome   10852 non-null  object
dtypes: int64(8), object(9)
memory usage: 1.4+ MB


In [38]:
##### 查看数值特征的统计特征，均值、方差、最小、最大等
train.describe()

,ID,age,balance,day,duration,campaign,pdays,previous,y
count,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000,25317.000000
mean,12659.000000,40.935379,1357.555082,15.835289,257.732393,2.772050,40.248766,0.591737,0.116957
std,7308.532719,10.634289,2999.822811,8.319480,256.975151,3.136097,100.213541,2.568313,0.321375
min,1.000000,18.000000,-8019.000000,1.000000,0.000000,1.000000,-1.000000,0.000000,0.000000
25%,6330.000000,33.000000,73.000000,8.000000,103.000000,1.000000,-1.000000,0.000000,0.000000
50%,12659.000000,39.000000,448.000000,16.000000,181.000000,2.000000,-1.000000,0.000000,0.000000
75%,18988.000000,48.000000,1435.000000,21.000000,317.000000,3.000000,-1.000000,0.000000,0.000000
max,25317.000000,95.000000,102127.000000,31.000000,3881.000000,55.000000,854.000000,275.000000,1.000000


In [39]:
##### 查看不同数值特征之间的相关性
train.corr()

,ID,age,balance,day,duration,campaign,pdays,previous,y
ID,1.000000,0.008465,0.032719,-0.020171,0.229149,-0.038265,0.065807,0.047028,0.556627
age,0.008465,1.000000,0.093740,-0.016070,0.000416,0.006171,-0.026431,0.006575,0.029916
balance,0.032719,0.093740,1.000000,0.010245,0.026042,-0.010419,0.001032,0.015792,0.057564
day,-0.020171,-0.016070,0.010245,1.000000,-0.031946,0.168830,-0.092892,-0.050706,-0.031886
duration,0.229149,0.000416,0.026042,-0.031946,1.000000,-0.087780,0.000040,0.001315,0.394746
campaign,-0.038265,0.006171,-0.010419,0.168830,-0.087780,1.000000,-0.089224,-0.031667,-0.075173
pdays,0.065807,-0.026431,0.001032,-0.092892,0.000040,-0.089224,1.000000,0.411688,0.107565
previous,0.047028,0.006575,0.015792,-0.050706,0.001315,-0.031667,0.411688,1.000000,0.088337
y,0.556627,0.029916,0.057564,-0.031886,0.394746,-0.075173,0.107565,0.088337,1.000000


In [40]:
#### 查看特定特征的取值分布
train.job.value_counts()

blue-collar      5456
management       5296
technician       4241
admin.           2909
services         2342
retired          1273
self-employed     884
entrepreneur      856
unemployed        701
housemaid         663
student           533
unknown           163
Name: job, dtype: int64

In [41]:
##### 数据的类别分布
train.y.value_counts()

0    22356
1     2961
Name: y, dtype: int64

> - 数据集中，需要注意不同的数据格式，`object，int64`等
- 测试数据集中没有目标那一列，为了便于特征工程，通常**将训练数据与测试数据合并**

In [42]:
####### 将训练数据与测试数据合并
test["y"] = -1
data = train.append(test).reset_index(drop=True) # 重排索引
# data = pd.concat([train,test]).reset_index(drop=True)
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


## 特征工程

In [45]:
import os
import json
from numba import jit
from tqdm import tqdm
from tqdm import tqdm_notebook

import numpy as np
import pandas as pd

# about sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler as std
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import f1_score
#about time
import time
import datetime
from datetime import datetime, timedelta

#Garbage collection
import gc
# scipy
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats
import scipy.spatial.distance as dist
#other
from collections import Counter
from statistics import mode
#warning
import warnings
warnings.filterwarnings("ignore")
import json
import math
from itertools import product
import ast

In [46]:
############ 非数值特征编码

cat_col = [
    i for i in data.select_dtypes(object).columns if i not in ['ID', 'y']
]
for i in tqdm_notebook(cat_col):
    lbl = LabelEncoder()
    # 转化为类别
    data[i] = lbl.fit_transform(data[i].astype(str))
    # 增加新特征，计算每种取值的数量
    data['count_' + i] = data.groupby([i])[i].transform('count')

In [47]:
data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,y,count_job,count_marital,count_education,count_default,count_housing,count_loan,count_contact,count_month,count_poutcome
0,1,43,4,1,2,0,291,1,0,2,...,0,7595,21857,10654,35524,19959,30363,10393,10957,29554
1,2,42,9,0,0,0,5076,1,0,0,...,0,6083,4140,5493,35524,19959,30363,23437,2365,1475
2,3,47,0,1,1,0,104,1,1,0,...,0,4124,21857,18509,35524,19959,5806,23437,5569,29554
3,4,28,4,2,1,0,-994,1,1,0,...,0,7595,10172,18509,35524,19959,5806,23437,5569,29554
4,5,42,9,0,1,0,2974,1,0,2,...,0,6083,4140,18509,35524,19959,30363,10393,10957,29554


In [49]:
feats = [col for col in data.columns if col not in ['y', 'ID']]
print(feats)

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'count_job', 'count_marital', 'count_education', 'count_default', 'count_housing', 'count_loan', 'count_contact', 'count_month', 'count_poutcome']


## 分类模型

### 最基础模型

In [55]:
model = lgb.LGBMClassifier(
    boosting_type='gbdt',
    num_leaves=30,
    reg_alpha=0,
    reg_lambda=0,
    n_estimators=1500,
    objective='binary',
    metric="auc",
    subsample=0.95,
    colsample_bytree=0.7,
    subsample_freq=1,
    learning_rate=0.2,
    random_state=2020,
)

train_x = data[data['y'] != -1][feats]
train_y = data[data['y'] != -1]['y']
test_x = data[data['y'] == -1][feats]

train_x, val_x, train_y, val_y = train_test_split(
    train_x,
    train_y,
    test_size=0.3,
    random_state=42,
)

model.fit(
    train_x,
    train_y,
    eval_set=[(val_x, val_y)],
    eval_metric="auc",
    early_stopping_rounds=100,
)
test_pred = model.predict_proba(test_x)[:, 1]

[1]	valid_0's auc: 0.882792
Training until validation scores don't improve for 100 rounds
[2]	valid_0's auc: 0.900334
[3]	valid_0's auc: 0.907919
[4]	valid_0's auc: 0.919039
[5]	valid_0's auc: 0.922838
[6]	valid_0's auc: 0.923926
[7]	valid_0's auc: 0.924013
[8]	valid_0's auc: 0.926266
[9]	valid_0's auc: 0.92802
[10]	valid_0's auc: 0.927792
[11]	valid_0's auc: 0.929748
[12]	valid_0's auc: 0.929966
[13]	valid_0's auc: 0.930611
[14]	valid_0's auc: 0.931156
[15]	valid_0's auc: 0.931509
[16]	valid_0's auc: 0.931247
[17]	valid_0's auc: 0.933151
[18]	valid_0's auc: 0.933398
[19]	valid_0's auc: 0.933606
[20]	valid_0's auc: 0.934439
[21]	valid_0's auc: 0.934698
[22]	valid_0's auc: 0.934954
[23]	valid_0's auc: 0.934926
[24]	valid_0's auc: 0.934756
[25]	valid_0's auc: 0.934782
[26]	valid_0's auc: 0.934728
[27]	valid_0's auc: 0.935008
[28]	valid_0's auc: 0.935103
[29]	valid_0's auc: 0.935133
[30]	valid_0's auc: 0.935262
[31]	valid_0's auc: 0.935066
[32]	valid_0's auc: 0.935316
[33]	valid_0's auc: 

In [56]:
pred = data[data['y'] == -1][['ID']]
pred['pred'] = test_pred

In [57]:
pred.head()

,ID,pred
25317,25318,0.040618
25318,25319,0.018166
25319,25320,0.004880
25320,25321,0.599200
25321,25322,0.058148


### 交叉验证

In [59]:
from sklearn.model_selection import KFold
n_splits = 10
kfold = KFold(n_splits=10, shuffle=True, random_state=42)

train_x = data[data['y'] != -1][feats]
train_y = data[data['y'] != -1]['y']

res = data[data['y'] == -1][['ID']]
test_x = data[data['y'] == -1][feats]
res['pred'] = 0

for train_idx, val_idx in kfold.split(train_x):
    model.random_state = model.random_state + 1

    # 训练集索引，验证集索引
    train_x1 = train_x.loc[train_idx]
    train_y1 = train_y.loc[train_idx]

    val_x1 = train_x.loc[val_idx]
    val_y1 = train_y.loc[val_idx]

    # 模型拟合
    model.fit(train_x1,
              train_y1,
              eval_set=[(train_x1, train_y1), (val_x1, val_y1)])

    # 模型预测
    res['pred'] += model.predict_proba(test_x)[:, 1]

res['pred'] = res['pred'] / 10

## 结果保存到本地
# res.to_csv('./submission10_fold.csv', index=False)

[1]	training's auc: 0.770676	valid_1's auc: 0.766114
[2]	training's auc: 0.793711	valid_1's auc: 0.790743
[3]	training's auc: 0.801404	valid_1's auc: 0.784552
[4]	training's auc: 0.92015	valid_1's auc: 0.907589
[5]	training's auc: 0.930803	valid_1's auc: 0.917609
[6]	training's auc: 0.934361	valid_1's auc: 0.918405
[7]	training's auc: 0.938241	valid_1's auc: 0.923064
[8]	training's auc: 0.940171	valid_1's auc: 0.925207
[9]	training's auc: 0.941514	valid_1's auc: 0.927533
[10]	training's auc: 0.943292	valid_1's auc: 0.929315
[11]	training's auc: 0.944095	valid_1's auc: 0.928978
[12]	training's auc: 0.945465	valid_1's auc: 0.931375
[13]	training's auc: 0.946381	valid_1's auc: 0.931256
[14]	training's auc: 0.94774	valid_1's auc: 0.93204
[15]	training's auc: 0.948457	valid_1's auc: 0.932856
[16]	training's auc: 0.949524	valid_1's auc: 0.933375
[17]	training's auc: 0.950195	valid_1's auc: 0.932777
[18]	training's auc: 0.951011	valid_1's auc: 0.9327
[19]	training's auc: 0.952026	valid_1's au

[236]	training's auc: 0.998682	valid_1's auc: 0.926185
[237]	training's auc: 0.998704	valid_1's auc: 0.926171
[238]	training's auc: 0.998714	valid_1's auc: 0.926074
[239]	training's auc: 0.99873	valid_1's auc: 0.926126
[240]	training's auc: 0.99877	valid_1's auc: 0.92602
[241]	training's auc: 0.998799	valid_1's auc: 0.925777
[242]	training's auc: 0.998848	valid_1's auc: 0.925597
[243]	training's auc: 0.998862	valid_1's auc: 0.925671
[244]	training's auc: 0.99891	valid_1's auc: 0.925606
[245]	training's auc: 0.998919	valid_1's auc: 0.925731
[246]	training's auc: 0.998987	valid_1's auc: 0.92558
[247]	training's auc: 0.999002	valid_1's auc: 0.925587
[248]	training's auc: 0.99904	valid_1's auc: 0.925557
[249]	training's auc: 0.999064	valid_1's auc: 0.92541
[250]	training's auc: 0.999093	valid_1's auc: 0.925317
[251]	training's auc: 0.999135	valid_1's auc: 0.925373
[252]	training's auc: 0.999144	valid_1's auc: 0.925474
[253]	training's auc: 0.999155	valid_1's auc: 0.925627
[254]	training's 

[479]	training's auc: 1	valid_1's auc: 0.925438
[480]	training's auc: 1	valid_1's auc: 0.925441
[481]	training's auc: 1	valid_1's auc: 0.925485
[482]	training's auc: 1	valid_1's auc: 0.925582
[483]	training's auc: 1	valid_1's auc: 0.925621
[484]	training's auc: 1	valid_1's auc: 0.925688
[485]	training's auc: 1	valid_1's auc: 0.925736
[486]	training's auc: 1	valid_1's auc: 0.92569
[487]	training's auc: 1	valid_1's auc: 0.925808
[488]	training's auc: 1	valid_1's auc: 0.925819
[489]	training's auc: 1	valid_1's auc: 0.925792
[490]	training's auc: 1	valid_1's auc: 0.925647
[491]	training's auc: 1	valid_1's auc: 0.92578
[492]	training's auc: 1	valid_1's auc: 0.92569
[493]	training's auc: 1	valid_1's auc: 0.925758
[494]	training's auc: 1	valid_1's auc: 0.925772
[495]	training's auc: 1	valid_1's auc: 0.925856
[496]	training's auc: 1	valid_1's auc: 0.9259
[497]	training's auc: 1	valid_1's auc: 0.925946
[498]	training's auc: 1	valid_1's auc: 0.926021
[499]	training's auc: 1	valid_1's auc: 0.9258

[722]	training's auc: 1	valid_1's auc: 0.925351
[723]	training's auc: 1	valid_1's auc: 0.925395
[724]	training's auc: 1	valid_1's auc: 0.925334
[725]	training's auc: 1	valid_1's auc: 0.925332
[726]	training's auc: 1	valid_1's auc: 0.925338
[727]	training's auc: 1	valid_1's auc: 0.92549
[728]	training's auc: 1	valid_1's auc: 0.925553
[729]	training's auc: 1	valid_1's auc: 0.925593
[730]	training's auc: 1	valid_1's auc: 0.925633
[731]	training's auc: 1	valid_1's auc: 0.925515
[732]	training's auc: 1	valid_1's auc: 0.925582
[733]	training's auc: 1	valid_1's auc: 0.925572
[734]	training's auc: 1	valid_1's auc: 0.925513
[735]	training's auc: 1	valid_1's auc: 0.925519
[736]	training's auc: 1	valid_1's auc: 0.925356
[737]	training's auc: 1	valid_1's auc: 0.92545
[738]	training's auc: 1	valid_1's auc: 0.925465
[739]	training's auc: 1	valid_1's auc: 0.9255
[740]	training's auc: 1	valid_1's auc: 0.925524
[741]	training's auc: 1	valid_1's auc: 0.925487
[742]	training's auc: 1	valid_1's auc: 0.925

[963]	training's auc: 1	valid_1's auc: 0.924518
[964]	training's auc: 1	valid_1's auc: 0.924499
[965]	training's auc: 1	valid_1's auc: 0.924531
[966]	training's auc: 1	valid_1's auc: 0.924547
[967]	training's auc: 1	valid_1's auc: 0.92459
[968]	training's auc: 1	valid_1's auc: 0.924605
[969]	training's auc: 1	valid_1's auc: 0.924608
[970]	training's auc: 1	valid_1's auc: 0.924487
[971]	training's auc: 1	valid_1's auc: 0.924349
[972]	training's auc: 1	valid_1's auc: 0.92435
[973]	training's auc: 1	valid_1's auc: 0.924406
[974]	training's auc: 1	valid_1's auc: 0.924427
[975]	training's auc: 1	valid_1's auc: 0.924406
[976]	training's auc: 1	valid_1's auc: 0.924455
[977]	training's auc: 1	valid_1's auc: 0.924502
[978]	training's auc: 1	valid_1's auc: 0.924327
[979]	training's auc: 1	valid_1's auc: 0.924387
[980]	training's auc: 1	valid_1's auc: 0.924375
[981]	training's auc: 1	valid_1's auc: 0.924497
[982]	training's auc: 1	valid_1's auc: 0.924493
[983]	training's auc: 1	valid_1's auc: 0.9

...

[350]	training's auc: 0.999905	valid_1's auc: 0.918941
[351]	training's auc: 0.999905	valid_1's auc: 0.918945
[352]	training's auc: 0.99991	valid_1's auc: 0.918886
[353]	training's auc: 0.999913	valid_1's auc: 0.918944
[354]	training's auc: 0.999916	valid_1's auc: 0.918864
[355]	training's auc: 0.999917	valid_1's auc: 0.918968
[356]	training's auc: 0.999917	valid_1's auc: 0.919182
[357]	training's auc: 0.999918	valid_1's auc: 0.919378
[358]	training's auc: 0.999919	valid_1's auc: 0.919345
[359]	training's auc: 0.999923	valid_1's auc: 0.919328
[360]	training's auc: 0.999924	valid_1's auc: 0.919178
[361]	training's auc: 0.999927	valid_1's auc: 0.919338
[362]	training's auc: 0.999927	valid_1's auc: 0.919479
[363]	training's auc: 0.999929	valid_1's auc: 0.919694
[364]	training's auc: 0.99993	valid_1's auc: 0.919712
[365]	training's auc: 0.999932	valid_1's auc: 0.919934
[366]	training's auc: 0.999933	valid_1's auc: 0.919904
[367]	training's auc: 0.999935	valid_1's auc: 0.92004
[368]	trainin

[592]	training's auc: 1	valid_1's auc: 0.919693
[593]	training's auc: 1	valid_1's auc: 0.919723
[594]	training's auc: 1	valid_1's auc: 0.919642
[595]	training's auc: 1	valid_1's auc: 0.919653
[596]	training's auc: 1	valid_1's auc: 0.91959
[597]	training's auc: 1	valid_1's auc: 0.919576
[598]	training's auc: 1	valid_1's auc: 0.919691
[599]	training's auc: 1	valid_1's auc: 0.919634
[600]	training's auc: 1	valid_1's auc: 0.91951
[601]	training's auc: 1	valid_1's auc: 0.919561
[602]	training's auc: 1	valid_1's auc: 0.919757
[603]	training's auc: 1	valid_1's auc: 0.919714
[604]	training's auc: 1	valid_1's auc: 0.919538
[605]	training's auc: 1	valid_1's auc: 0.91952
[606]	training's auc: 1	valid_1's auc: 0.919471
[607]	training's auc: 1	valid_1's auc: 0.919367
[608]	training's auc: 1	valid_1's auc: 0.919449
[609]	training's auc: 1	valid_1's auc: 0.919558
[610]	training's auc: 1	valid_1's auc: 0.919507
[611]	training's auc: 1	valid_1's auc: 0.919666
[612]	training's auc: 1	valid_1's auc: 0.91

[831]	training's auc: 1	valid_1's auc: 0.918091
[832]	training's auc: 1	valid_1's auc: 0.918135
[833]	training's auc: 1	valid_1's auc: 0.918223
[834]	training's auc: 1	valid_1's auc: 0.918288
[835]	training's auc: 1	valid_1's auc: 0.918293
[836]	training's auc: 1	valid_1's auc: 0.918332
[837]	training's auc: 1	valid_1's auc: 0.918417
[838]	training's auc: 1	valid_1's auc: 0.918435
[839]	training's auc: 1	valid_1's auc: 0.918436
[840]	training's auc: 1	valid_1's auc: 0.918553
[841]	training's auc: 1	valid_1's auc: 0.918546
[842]	training's auc: 1	valid_1's auc: 0.918635
[843]	training's auc: 1	valid_1's auc: 0.918635
[844]	training's auc: 1	valid_1's auc: 0.918578
[845]	training's auc: 1	valid_1's auc: 0.918655
[846]	training's auc: 1	valid_1's auc: 0.918539
[847]	training's auc: 1	valid_1's auc: 0.9186
[848]	training's auc: 1	valid_1's auc: 0.91864
[849]	training's auc: 1	valid_1's auc: 0.918618
[850]	training's auc: 1	valid_1's auc: 0.91861
[851]	training's auc: 1	valid_1's auc: 0.918

[1070]	training's auc: 1	valid_1's auc: 0.919673
[1071]	training's auc: 1	valid_1's auc: 0.919736
[1072]	training's auc: 1	valid_1's auc: 0.919643
[1073]	training's auc: 1	valid_1's auc: 0.919672
[1074]	training's auc: 1	valid_1's auc: 0.919492
[1075]	training's auc: 1	valid_1's auc: 0.919467
[1076]	training's auc: 1	valid_1's auc: 0.919499
[1077]	training's auc: 1	valid_1's auc: 0.919461
[1078]	training's auc: 1	valid_1's auc: 0.919468
[1079]	training's auc: 1	valid_1's auc: 0.919457
[1080]	training's auc: 1	valid_1's auc: 0.919606
[1081]	training's auc: 1	valid_1's auc: 0.91962
[1082]	training's auc: 1	valid_1's auc: 0.919583
[1083]	training's auc: 1	valid_1's auc: 0.919552
[1084]	training's auc: 1	valid_1's auc: 0.919478
[1085]	training's auc: 1	valid_1's auc: 0.919499
[1086]	training's auc: 1	valid_1's auc: 0.919433
[1087]	training's auc: 1	valid_1's auc: 0.919442
[1088]	training's auc: 1	valid_1's auc: 0.919514
[1089]	training's auc: 1	valid_1's auc: 0.919492
[1090]	training's auc

[1310]	training's auc: 1	valid_1's auc: 0.918959
[1311]	training's auc: 1	valid_1's auc: 0.919045
[1312]	training's auc: 1	valid_1's auc: 0.919078
[1313]	training's auc: 1	valid_1's auc: 0.91912
[1314]	training's auc: 1	valid_1's auc: 0.919134
[1315]	training's auc: 1	valid_1's auc: 0.919092
[1316]	training's auc: 1	valid_1's auc: 0.919113
[1317]	training's auc: 1	valid_1's auc: 0.919071
[1318]	training's auc: 1	valid_1's auc: 0.918961
[1319]	training's auc: 1	valid_1's auc: 0.919
[1320]	training's auc: 1	valid_1's auc: 0.918983
[1321]	training's auc: 1	valid_1's auc: 0.918951
[1322]	training's auc: 1	valid_1's auc: 0.918962
[1323]	training's auc: 1	valid_1's auc: 0.91894
[1324]	training's auc: 1	valid_1's auc: 0.918895
[1325]	training's auc: 1	valid_1's auc: 0.918794
[1326]	training's auc: 1	valid_1's auc: 0.91884
[1327]	training's auc: 1	valid_1's auc: 0.918799
[1328]	training's auc: 1	valid_1's auc: 0.918728
[1329]	training's auc: 1	valid_1's auc: 0.918771
[1330]	training's auc: 1	v